In [9]:
import os
os.environ['NUMEXPR_MAX_THREADS'] = '1'

import logging
import numexpr as ne
import numpy as np
import torch
import datetime
from ddopai.envs.pricing.dynamic import DynamicPricingEnv
from ddopai.envs.pricing.dynamic_inventory import DynamicPricingInvEnv
from ddopai.envs.actionprocessors import ClipAction, RoundAction

from ddopai.experiments.experiment_functions_online import run_experiment
from ddopai.experiments.meta_experiment_functions import *
import requests
import yaml
import re
import pandas as pd
import wandb
from copy import deepcopy
import warnings
import gc
from mushroom_rl import core 
import pickle

In [10]:
logging_level = logging.INFO
logging.basicConfig(level=logging_level)

ne.set_num_threads(1)
torch.backends.cudnn.enabled = False
torch.set_num_threads(1)

set_warnings(logging.INFO) # turn off warnings for any level higher or equal to the input level

PROJECT_NAME = "pricing_cMDP_test"
LIBRARIES_TO_TRACK = ["ddopai", "mushroom_rl"]
ENVCLASS = DynamicPricingEnv
RESULTS_DIR = "results"
def get_ENVCLASS(class_name):
    if class_name == "DynamicPricingEnv":
        return DynamicPricingEnv
    elif class_name == "DynamicPricingInvEnv":
        return DynamicPricingInvEnv
    else:
        raise ValueError(f"Unknown class name {class_name}")

# Experiment preparations
## Set-up WandB
### Init WandB

In [11]:
project_name = "pricing_cMDP"

run = wandb.init(
    project=project_name,
    name = f"{project_name}_artifact_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
)

### Track library versions and git hash of experiment

# Experiment parameters

In [ ]:
config_train, config_agent, config_env, AgentClass, agent_name = prep_experiment(
        PROJECT_NAME,
        LIBRARIES_TO_TRACK,
        config_train_name="config_train.yaml",
        config_agent_name="config_agent.yaml",
        config_env_name="config_env.yaml",
    )

SettingsValidationError: Invalid value for property project: ['ddopai', 'mushroom_rl'].

## Prepare data
* Get the env parameters 
* Create the data and noise based on these parameters

In [5]:
raw_data, val_index_start, test_index_start = get_online_data(
        config_env,
        overwrite=False
    )
# TODO: add as a wandb Artifact

## Log Data as Wandb Artifact

In [ ]:
with open('data/raw_data.pkl', 'wb') as f:
    pickle.dump(raw_data, f)

In [7]:
artifact = wandb.Artifact('raw_data', type='data')
artifact.add_file('data/raw_data.pkl')
wandb.log_artifact(artifact)
wandb.finish()